In [0]:
from google.colab import drive
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
from nltk.stem.snowball import SnowballStemmer
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
stemmer = SnowballStemmer("english")

In [0]:
from tqdm import tqdm

import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd

stemmed_data_file = "stemmed_data.parquet"
stop_words = set(stopwords.words('english'))
for i, df in tqdm(enumerate(pd.read_csv("drive/My Drive/WashingtonPost.v2/data/tokenized_data.csv", chunksize=10_000, header=None)), total=60):
    df[2] = df[2].map(lambda x: x[1:-1]).map(lambda x: [ stemmer.stem(xi.strip()[1:-1]) for xi in str.split(x, sep=',') if xi.strip()[1:-1] not in stop_words])

    table = pa.Table.from_pandas(df)
    # for the first chunk of records
    if i == 0:
        # create a parquet write object giving it an output file
        pqwriter = pq.ParquetWriter(stemmed_data_file, table.schema)
        pqwriter.write_table(table)
    # subsequent chunks can be written to the same file
    else:
        pqwriter.write_table(table)
if pqwriter:
    pqwriter.close()

Move file to Google Drive

In [0]:
!ls -lh

total 881M
drwx------ 4 root root 4.0K Dec 19 11:41 drive
drwxr-xr-x 1 root root 4.0K Dec 12 16:48 sample_data
-rw-r--r-- 1 root root 881M Dec 19 14:19 stemmed_data.parquet


In [0]:
!mv 'stemmed_data.parquet' 'drive/My Drive/processed/'

In [0]:
pathfile = "/content/drive/My Drive/processed/stemmed_data.parquet"
stemmed_df = pd.read_parquet(pathfile)